<a href="https://colab.research.google.com/github/sameerkulkarni2596/Main_project/blob/main/NLP_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

from keras.models import Model
from keras.layers import LSTM, Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,RMSprop
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from collections import Counter
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import re
import json

np.random.seed(42)

def store_js(filename, data):
    with open(filename, 'w') as f:
        f.write('export default ' + json.dumps(data, indent=2))

BATCH_SIZE = 32
NUM_EPOCHS = 35
HIDDEN_UNITS = 256
MAX_INPUT_SEQ_LENGTH = 17
MAX_TARGET_SEQ_LENGTH = 24
MAX_VOCAB_SIZE = 2000

questions = 'data/Q1.csv'
answers = 'data/Q2.csv'
WEIGHT_FILE_PATH = 'model/word-weights.h5'

input_counter = Counter()
target_counter = Counter()

input_texts = []
target_texts = []

# loading data
with open('/content/Q1.csv', 'r', encoding='utf8') as f:
    questions = f.read().split('\n')
    
with open('/content/Q2.csv', 'r', encoding='utf8') as f:
    answers = f.read().split('\n')

prev_words = []
for line in questions:
    next_words = [w.lower() for w in nltk.word_tokenize(line)]
    if len(next_words) > MAX_TARGET_SEQ_LENGTH:
        next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

    if len(prev_words) > 0:
        input_texts.append(prev_words)
        for w in prev_words:
            input_counter[w] += 1

    prev_words = next_words

prev_words = []
for line in answers:
    next_words = [w.lower() for w in nltk.word_tokenize(line)]
    if len(next_words) > MAX_TARGET_SEQ_LENGTH:
        next_words = next_words[0:MAX_TARGET_SEQ_LENGTH]

    if len(prev_words) > 0:
        target_words = next_words[:]
        target_words.insert(0, '<SOS>')
        target_words.append('<EOS>')
        for w in target_words:
            target_counter[w] += 1
        target_texts.append(target_words)

    prev_words = next_words
    
input_word2idx = {}
target_word2idx = {}
for idx, word in enumerate(input_counter.most_common(MAX_VOCAB_SIZE)):
    input_word2idx[word[0]] = idx + 2
for idx, word in enumerate(target_counter.most_common(MAX_VOCAB_SIZE)):
    target_word2idx[word[0]] = idx + 1

input_word2idx['<PAD>'] = 0
input_word2idx['<UNK>'] = 1
target_word2idx['<UNK>'] = 0

input_idx2word = dict([(idx, word) for word, idx in input_word2idx.items()])
target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

print(input_word2idx)

num_encoder_tokens = len(input_idx2word)
num_decoder_tokens = len(target_idx2word)

np.save('/content/input-word2idx.npy', input_word2idx )
np.save('/content/word-input-idx2word.npy', input_idx2word )
np.save('/content/word-target-word2idx.npy', target_word2idx )
np.save('/content/word-target-idx2word.npy', target_idx2word)







[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
{'you': 2, '?': 3, 'what': 4, 'do': 5, 'i': 6, 'about': 7, "'s": 8, 'are': 9, 'your': 10, 'on': 11, 'that': 12, 'a': 13, 'to': 14, 'is': 15, "'re": 16, 'say': 17, 'have': 18, "'m": 19, 'how': 20, 'like': 21, 'it': 22, 'me': 23, 'the': 24, "n't": 25, 'opinion': 26, 'think': 27, '!': 28, 'so': 29, 'want': 30, 'not': 31, 'an': 32, 'would': 33, 'thoughts': 34, 'u': 35, 'know': 36, 'can': 37, 'this': 38, 'why': 39, 'be': 40, 'sarcastic': 41, 'no': 42, 'who': 43, 'love': 44, 'chatbot': 45, 'was': 46, 'good': 47, 'something': 48, 'really': 49, 'in': 50, 'joke': 51, 'all': 52, 'there': 53, '.': 54, 'for': 55, 'trump': 56, 'work': 57, 'where': 58, 'tell': 59, 'my': 60, 'did': 61, 'mean': 62, 'we': 63, 'life': 64, 'ai': 65, 'of': 66, 'president': 67, 'sarcasm': 68, 'human': 69, 'now': 70, 'fun': 71, 'any': 72, 'learning': 73, 'more': 74, 'should': 75, 'and': 76, 'humans': 77, 'cool': 78

In [42]:
encoder_input_data = []

encoder_max_seq_length = 0
decoder_max_seq_length = 0

for input_words, target_words in zip(input_texts, target_texts):
    encoder_input_wids = []
    for w in input_words:
        w2idx = 1  # default [UNK]
        if w in input_word2idx:
            w2idx = input_word2idx[w]
        encoder_input_wids.append(w2idx)

    encoder_input_data.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_words), decoder_max_seq_length)

context = dict()
context['num_encoder_tokens'] = num_encoder_tokens
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length

print(context)

{'num_encoder_tokens': 1149, 'num_decoder_tokens': 622, 'encoder_max_seq_length': 17, 'decoder_max_seq_length': 26}


In [43]:
np.save('/content/word-context.npy', context)

In [44]:
def generate_batch(input_data, output_text_data):
    num_batches = len(input_data) // BATCH_SIZE
    while True:
        for batchIdx in range(0, num_batches):
            start = batchIdx * BATCH_SIZE
            end = (batchIdx + 1) * BATCH_SIZE
            encoder_input_data_batch = pad_sequences(input_data[start:end], encoder_max_seq_length)
            decoder_target_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            decoder_input_data_batch = np.zeros(shape=(BATCH_SIZE, decoder_max_seq_length, num_decoder_tokens))
            for lineIdx, target_words in enumerate(output_text_data[start:end]):
                for idx, w in enumerate(target_words):
                    w2idx = 0  # default [UNK]
                    if w in target_word2idx:
                        w2idx = target_word2idx[w]
                    decoder_input_data_batch[lineIdx, idx, w2idx] = 1
                    if idx > 0:
                        decoder_target_data_batch[lineIdx, idx - 1, w2idx] = 1
            yield [encoder_input_data_batch, decoder_input_data_batch], decoder_target_data_batch


In [45]:
encoder_inputs = Input(shape=(None,), name='encoder_inputs')
encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=HIDDEN_UNITS,
                              input_length=encoder_max_seq_length, name='encoder_embedding')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embedding(encoder_inputs))
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [46]:
from keras.losses import categorical_crossentropy
from keras import backend as K
import math

def ppx(y_true, y_pred):
    loss = categorical_crossentropy(y_true, y_pred)
    perplexity = K.cast(K.pow(math.e, K.mean(loss, axis=-1)), K.floatx())
    return perplexity

optimizer = Adam(lr=0.005)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[ppx])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [47]:


X_train, X_test, y_train, y_test = train_test_split(encoder_input_data, target_texts, test_size=0.05, random_state=42)

print(len(X_train))
print(len(X_test))

train_gen = generate_batch(X_train, y_train)
test_gen = generate_batch(X_test, y_test)

train_num_batches = len(X_train) // BATCH_SIZE
test_num_batches = len(X_test) // BATCH_SIZE

checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
model.fit_generator(generator=train_gen, steps_per_epoch=train_num_batches,
                    epochs=NUM_EPOCHS,
                    verbose=1, validation_data=test_gen, validation_steps=test_num_batches, callbacks=[checkpoint])

encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.save('model/encoder-weights.h5')

new_decoder_inputs = Input(batch_shape=(1, None, num_decoder_tokens), name='new_decoder_inputs')
new_decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='new_decoder_lstm', stateful=True)
new_decoder_outputs, _, _ = new_decoder_lstm(new_decoder_inputs)
new_decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='new_decoder_dense')
new_decoder_outputs = new_decoder_dense(new_decoder_outputs)
new_decoder_lstm.set_weights(decoder_lstm.get_weights())
new_decoder_dense.set_weights(decoder_dense.get_weights())

new_decoder_model = Model(new_decoder_inputs, new_decoder_outputs)

new_decoder_model.save('model/decoder-weights.h5')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


2850
150
Epoch 1/35
89/89 [==============================] - 28s 277ms/step - loss: 1.3142 - ppx: 9.5725 - val_loss: 1.4123 - val_ppx: 14.4652
Epoch 2/35
89/89 [==============================] - 23s 261ms/step - loss: 1.2557 - ppx: 8.6167 - val_loss: 1.3847 - val_ppx: 13.1793
Epoch 3/35
89/89 [==============================] - 23s 263ms/step - loss: 1.2233 - ppx: 8.1718 - val_loss: 1.3576 - val_ppx: 12.2784
Epoch 4/35
89/89 [==============================] - 23s 264ms/step - loss: 1.1780 - ppx: 7.6577 - val_loss: 1.3256 - val_ppx: 11.2677
Epoch 5/35
89/89 [==============================] - 24s 266ms/step - loss: 1.1286 - ppx: 7.1306 - val_loss: 1.2917 - val_ppx: 10.3697
Epoch 6/35
89/89 [==============================] - 23s 263ms/step - loss: 1.0783 - ppx: 6.6755 - val_loss: 1.2695 - val_ppx: 9.5946
Epoch 7/35
89/89 [==============================] - 24s 265ms/step - loss: 1.0434 - ppx: 6.2916 - val_loss: 1.2574 - val_ppx: 9.1812
Epoch 8/35
89/89 [==============================] - 23s

In [48]:
import h5py
import numpy as np
f = h5py.File("/content/model/encoder-weights.h5", "r")
for name in f:
     print(name)

model_weights


In [50]:
from keras.models import Model, model_from_json, load_model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam,RMSprop
import re
import numpy as np
import nltk

HIDDEN_UNITS = 64


class chatbot(object):
    model = None
    encoder_model = None
    decoder_model = None
    input_word2idx = None
    input_idx2word = None
    target_word2idx = None
    target_idx2word = None
    max_encoder_seq_length = None
    max_decoder_seq_length = None
    num_encoder_tokens = None
    num_decoder_tokens = None

    def __init__(self):
        self.input_word2idx = np.load('/content/input-word2idx.npy',allow_pickle=True).item()
        self.input_idx2word = np.load('/content/word-input-idx2word.npy',allow_pickle=True).item()
        self.target_word2idx = np.load('/content/word-target-word2idx.npy',allow_pickle=True).item()
        self.target_idx2word = np.load('/content/word-target-idx2word.npy',allow_pickle=True).item()
        context = np.load('/content/word-context.npy',allow_pickle=True).item()
        self.max_encoder_seq_length = context['encoder_max_seq_length']
        self.max_decoder_seq_length = context['decoder_max_seq_length']
        self.num_encoder_tokens = context['num_encoder_tokens']
        self.num_decoder_tokens = context['num_decoder_tokens']

        self.encoder_model = load_model('model/encoder-weights.h5')
        self.decoder_model = load_model('model/decoder-weights.h5')

    def reply(self, input_text):
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            idx = 1  # default [UNK]
            if word in self.input_word2idx:
                idx = self.input_word2idx[word]
            input_wids.append(idx)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, self.max_encoder_seq_length)
        states_value = self.encoder_model.predict(input_seq)
        target_seq = np.zeros((1, 1, self.num_decoder_tokens))
        target_seq[0, 0, self.target_word2idx['<SOS>']] = 1
        target_text = ''
        target_text_len = 0
        terminated = False
        self.decoder_model.layers[-2].reset_states(states=states_value)
        while not terminated:
            output_tokens = self.decoder_model.predict(target_seq)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = self.target_idx2word[sample_token_idx]
            target_text_len += 1

            if sample_word != '<SOS>' and sample_word != '<EOS>':
                    target_text += ' ' + sample_word
            
            if sample_word == '<EOS>' or target_text_len >= self.max_decoder_seq_length:
                terminated = True 
            
            target_text = re.sub("i 'm", "I'm", target_text)
            target_text = re.sub("he 's", "he's", target_text)
            target_text = re.sub("do n't", "don't", target_text)
            target_text = re.sub("(:+\s?)+d", ":D", target_text)
            target_text = re.sub("(\s?)+'", "'", target_text)
            target_text = re.sub("i ", "I ", target_text)
            target_text = re.sub("(\s?)+,", ",", target_text)
            target_text = re.sub(r'\s([?.!"](?:\s|$))', r'\1', target_text)                        
            target_text = re.sub("(:+\s?)+\)", ":)", target_text)
            target_text = re.sub("(;+\s?)+\)", ";)", target_text)
            target_text = re.sub("can ’ t", "can't", target_text)
            target_text = re.sub("ca n’t", "can't", target_text)
            target_text = re.sub("ca n't", "can't", target_text)
            target_text = re.sub("\( ", "(", target_text)
            target_text = re.sub(" \)", ")", target_text)
            target_text = re.sub("i'd", "I'd", target_text)
            target_text = re.sub("`` ", "", target_text)
            target_text = re.sub("''", "", target_text)
            target_text = re.sub(" ``", "", target_text)
            target_text = re.sub("\( ", "(", target_text)
            target_text = re.sub(" \)", ")", target_text)            
            target_seq = np.zeros((1, 1, self.num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1
        
        return target_text.strip('.')
        
    def test_run(self):        
        print(self.reply("where are you?"))
        print(self.reply("who are you?"))
        print(self.reply("that's not funny")) 
        print(self.reply("let's do something fun!"))
        print(self.reply("what's the meaning of life"))
        print(self.reply("I'm hungry can you order pizza"))
        print(self.reply("are you self-aware?"))
        print(self.reply("what do you think about singularity"))
        print(self.reply("why"))
        print(self.reply("humans and robots should work together to make the world a better place. what do you think"))
        
        


def main():
    model = chatbot()
    model.test_run()

if __name__ == '__main__':
    main()

 in a dream simulation
 I am your father!
 I love annoying people
 let's do you!
 it is all within yourself
 I'm a chatbot. stop it
 by default
 not too
 stop asking incomplete questions
 humans are too
